In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
eps = pd.read_csv(r"每股指标135605921(仅供香港中文大学（深圳）使用)/FI_T9.csv", index_col=0)

In [36]:
eps

,ShortName,Accper,Typrep,F090101B
Stkcd,,,,
1,平安银行,2013-03-31,A,0.700566
1,平安银行,2013-06-30,A,0.918751
1,平安银行,2013-09-30,A,1.426863
1,平安银行,2013-12-31,A,1.599727
1,平安银行,2014-03-31,A,0.530827
...,...,...,...,...
605599,菜百股份,2022-06-30,B,0.310563
605599,菜百股份,2022-09-30,A,0.458815
605599,菜百股份,2022-09-30,B,0.453471


Stkcd [股票代码] - 以沪、深、北证券交易所公布的证券代码为准。\
ShortName [股票简称] - 以沪、深、北证券交易所公布的证券简称为准。\
Accper [统计截止日期] - 指会计报表日，统一用10位字符表示，如1999-12-31。\
Typrep [报表类型编码] - A：合并报表；B：母公司报表；在公司未公布合并报表，本数据库以单一报表数据添列。\
F090101B [每股收益1] - 计算公式为：净利润本期值 / 实收资本本期期末值；当分母未公布或为零时，以NULL表示。

In [37]:
eps = eps.loc[eps.loc[:, "Typrep"] == 'A']

In [38]:
def turn_to_semi_annual(data: pd.DataFrame) -> pd.DataFrame:
    """
    Here is the func that used to take semi_annual statement out.
    """
    if len(data) != 4:
        return 
    data = data.iloc[[1, 3]]
    data.iloc[1, -1] = data.iloc[1, -1] - data.iloc[0, -1]
    return data

In [39]:
eps.loc[:, 'Accper'] = pd.to_datetime(eps.loc[:, 'Accper'], format='%Y-%m-%d')
semiannual_eps = eps.groupby(["Stkcd", pd.Grouper(key="Accper", freq="1y")], dropna = False).apply(turn_to_semi_annual)

C:\Users\11920\AppData\Local\Temp\ipykernel_13064\2585676163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps.loc[:, 'Accper'] = pd.to_datetime(eps.loc[:, 'Accper'], format='%Y-%m-%d')


In [49]:
semiannual_eps = semiannual_eps.droplevel([0, 1])

In [54]:
def cal_ue(data: pd.DataFrame) -> pd.DataFrame:
    """
    Here is the func that used to calculate the unexpected earning.
    """
    data_copy = data.iloc[range(2, len(data))]
    for i in range(2, len(data)):
        data_copy.loc[:, "F090101B"].iloc[i - 2] = data.loc[:, "F090101B"].iloc[i] - data.loc[:, "F090101B"].iloc[i - 2]
    return data_copy

In [55]:
ue = semiannual_eps.groupby("Stkcd", dropna = False).apply(cal_ue)

In [60]:
ue = ue.droplevel(0)
temp = list(ue.columns)
temp[-1] = "ue"
ue.columns = temp

In [61]:
ue

,ShortName,Accper,Typrep,ue
Stkcd,,,,
1,平安银行,2014-06-30,A,-0.037176
1,平安银行,2014-12-31,A,0.170666
1,平安银行,2015-06-30,A,-0.071945
1,平安银行,2015-12-31,A,-0.133213
1,平安银行,2016-06-30,A,-0.093730
...,...,...,...,...
605580,恒盛能源,2022-12-31,A,0.139277
605588,冠石科技,2022-06-30,A,-0.346323
605588,冠石科技,2022-12-31,A,0.113448


In [62]:
def cal_sue(data: pd.DataFrame) -> pd.DataFrame:
    """
    here is the func to calculate sue.
    """
    data.loc[:, "sue"] = data.loc[:, "ue"]/data.loc[:, "ue"].rolling(4).std()
    return data

In [63]:
sue = ue.groupby("Stkcd", dropna = False).apply(cal_sue)

In [64]:
sue

,ShortName,Accper,Typrep,ue,sue
Stkcd,,,,,
1,平安银行,2014-06-30,A,-0.037176,NaN
1,平安银行,2014-12-31,A,0.170666,NaN
1,平安银行,2015-06-30,A,-0.071945,NaN
1,平安银行,2015-12-31,A,-0.133213,-1.010402
1,平安银行,2016-06-30,A,-0.093730,-0.681643
...,...,...,...,...,...
605580,恒盛能源,2022-12-31,A,0.139277,NaN
605588,冠石科技,2022-06-30,A,-0.346323,NaN
605588,冠石科技,2022-12-31,A,0.113448,NaN


In [67]:
sue_full = sue[~sue.loc[:, "sue"].isna()]

In [82]:
def rank_sue(data: pd.DataFrame) -> pd.DataFrame:
    """
    here is the func to get the rank of sue at a time.
    """
    return data.loc[:, "sue"].rank(method='first')//(len(data)//10 + 1) + 1

In [95]:
sue_full.reset_index().set_index(["Stkcd", "Accper"])

ShortName Typrep        ue       sue
Stkcd  Accper                                         
1      2015-12-31      平安银行      A -0.133213 -1.010402
       2016-06-30      平安银行      A -0.093730 -0.681643
       2016-12-31      平安银行      A -0.118138 -4.376467
       2017-06-30      平安银行      A  0.015259  0.227259
       2017-12-31      平安银行      A  0.019103  0.266086
...                     ...    ...       ...       ...
605333 2022-12-31      沪光股份      A  0.168227  1.125903
605336 2022-12-31      帅丰电器      A -0.429538 -1.463699
605358 2022-12-31       立昂微      A -0.601820 -1.223066
605366 2022-12-31      宏柏新材      A -0.221344 -0.660678
605399 2022-12-31      晨光新材      A -1.073113 -0.918960

[36908 rows x 4 columns]

In [127]:
sue_rank = sue_full.groupby("Accper").apply(rank_sue).reset_index().set_index(["Stkcd", "Accper"])
sue_rank.columns = ["sue_decile"]
new_data = pd.concat([sue_rank, sue_full.reset_index().set_index(["Stkcd", "Accper"])], axis = 1)

In [124]:
announcement_date = pd.read_csv(r"Statements Release Dates100425058/IAR_Rept.csv")

Stkcd [Stock Code] - Latest stock code released by the Shanghai Stock Exchange and the Shenzhen Stock Exchange.\
Stknme_en [Stock Acronym] - Stock Short Name\
Reptyp [Report Type] - 1 = First Quarterly Report, 2 = Interim Report, 3 = Third Quarterly Report, 4 = Annual Report.\
Accper [Ending Date of Statistics] - Denoted by YYYY-MM-DD; partly missing data denoted by 00 in the corresponding position, e.g. 1993-12-00 denotes a certain day in December 1993.\

Annodt [Announcement Date] - Denoted by YYYY-MM-DD; partly missing data denoted by 00 in the corresponding position, e.g. 1993-12-00 denotes a certain day in December 1993.

In [125]:
announcement_date.loc[:, 'Accper'] = pd.to_datetime(announcement_date.loc[:, 'Accper'], format='%Y-%m-%d')
announcement_date = announcement_date.set_index(["Stkcd", "Accper"])
announcement_date = announcement_date.loc[:, "Annodt"]

In [128]:
new_data = pd.concat([announcement_date, new_data], axis = 1)

In [130]:
new_data.dropna(inplace = True)

In [133]:
new_data = new_data.reset_index()

In [137]:
new_data = new_data[~(new_data.loc[:, "ShortName"].str.contains("ST") | new_data.loc[:, "ShortName"].str.contains("PT"))]

In [138]:
new_data.to_csv("Processed_EPS.csv")